# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gmaps_key
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_data = pd.read_csv('city_data.csv')
city_data

,Unnamed: 0,City,Latitude,Longitude,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dwarka,22.24,68.97,84.47,53,0,13.09,IN,1603731853
1,1,port alfred,-33.59,26.89,65.35,83,100,28.61,ZA,1603731574
2,2,bara,10.37,10.73,89.60,22,11,5.82,NG,1603731856
3,3,zhuhai,22.28,113.57,78.01,83,0,13.87,CN,1603731856
4,5,lac-megantic,45.58,-70.88,37.00,74,100,1.99,CA,1603731856
...,...,...,...,...,...,...,...,...,...,...
945,1013,new ulm,44.31,-94.46,23.00,73,1,6.93,US,1603732059
946,1014,ottawa,45.41,-75.70,36.00,100,90,8.05,CA,1603731938
947,1016,kralendijk,12.15,-68.27,89.60,59,40,18.34,BQ,1603732001
948,1017,pacifica,37.61,-122.49,62.01,25,20,17.22,US,1603732121


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=gmaps_key)
locations = city_data[['Latitude','Longitude']]

In [19]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=city_data['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
ideal_cities = city_data.loc[city_data['Max temp'] < 90]
ideal_cities = ideal_cities.loc[ideal_cities['Max temp'] > 80]
ideal_cities = ideal_cities[ideal_cities['Wind Speed'] < 10]
ideal_cities = ideal_cities[ideal_cities['Cloudiness'] == 0]
ideal_cities

,Unnamed: 0,City,Latitude,Longitude,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,99,salalah,17.02,54.09,80.60,65,0,1.12,OM,1603731772
151,164,hithadhoo,-0.60,73.08,83.55,65,0,5.82,MV,1603731907
285,306,santiago del estero,-27.80,-64.26,84.36,14,0,7.16,AR,1603731653
420,448,elat,29.56,34.95,84.20,62,0,6.93,IL,1603731978
454,484,nokaneng,-19.67,22.27,87.58,12,0,9.46,BW,1603731989
470,502,clearwater,27.97,-82.80,88.00,66,0,6.93,US,1603731994
495,529,keti bandar,24.14,67.45,80.38,53,0,7.56,PK,1603731999
511,548,marsh harbour,26.54,-77.06,82.92,73,0,8.68,BS,1603731845
531,569,arlit,18.74,7.39,87.39,12,0,7.83,NE,1603732004
543,581,abu dhabi,24.47,54.37,84.20,51,0,5.82,AE,1603731905


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",
    "radius": '10000',
    "type": "hotel",
    "key": gmaps_key,
}
hotel_df = ideal_cities
hotel_df['Hotel Name'] = ''
for index, row in hotel_df.iterrows():
#       print(row['Latitude'])
    params['location'] = f"{row['Latitude']}, {row['Longitude']}"
    print(f"Retrieving results for {row['City']}")
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][1]['name']
    except:
        hotel_df.loc[index, 'Hotel Name'] = ''
hotel_df = hotel_df.rename(columns={'Latitude':'Lat','Longitude':'Lng'})

Retrieving results for salalah
Retrieving results for hithadhoo
Retrieving results for santiago del estero
Retrieving results for elat
Retrieving results for nokaneng
Retrieving results for clearwater
Retrieving results for keti bandar
Retrieving results for marsh harbour
Retrieving results for arlit
Retrieving results for abu dhabi
Retrieving results for mangrol
Retrieving results for buraydah
Retrieving results for doha
Retrieving results for keshod
Retrieving results for villa carlos paz
Retrieving results for tanout
Retrieving results for dakoro


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
92,99,salalah,17.02,54.09,80.60,65,0,1.12,OM,1603731772,Al Jabal
151,164,hithadhoo,-0.60,73.08,83.55,65,0,5.82,MV,1603731907,Scoop Guest House
285,306,santiago del estero,-27.80,-64.26,84.36,14,0,7.16,AR,1603731653,HOTEL URBANO
420,448,elat,29.56,34.95,84.20,62,0,6.93,IL,1603731978,Club Hotel Eilat
454,484,nokaneng,-19.67,22.27,87.58,12,0,9.46,BW,1603731989,Nokaneng Primary School
470,502,clearwater,27.97,-82.80,88.00,66,0,6.93,US,1603731994,Shephard's Beach Resort
495,529,keti bandar,24.14,67.45,80.38,53,0,7.56,PK,1603731999,EPI Center RHC Keti Bander
511,548,marsh harbour,26.54,-77.06,82.92,73,0,8.68,BS,1603731845,Abaco Beach Resort
531,569,arlit,18.74,7.39,87.39,12,0,7.83,NE,1603732004,مكن لشراء الذهب. محمد جبرين
543,581,abu dhabi,24.47,54.37,84.20,51,0,5.82,AE,1603731905,Emirates Palace


In [23]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
# Display figure
fig
# hotel_info

Figure(layout=FigureLayout(height='420px'))